# AI Search Pull Type

# 02_Chunking Optimization
Tips for Chunking Optimization to create search indexes for RAG

## Use case
- Azure AI Search の サービス仕様ドキュメントをインプットにする。
  - https://learn.microsoft.com/ja-jp/azure/search/
- ドキュメントは OCR が必要。
- 開発者マニュアルは、構造化されたセクションとなっている。
- 各セクションは非常に詳細かつ専門性の高い技術解説が記載されており、ドキュメントサイズも大きい。
- ドキュメントには、テキスト、テーブル、図、グラフなどが含まれるが、ここでは、テキスト、テーブルデータのみを扱う。

## チャンキング設計
- Document Intelligence で、Markdown形式でテキストデータを抽出済み。
- 1つのドキュメントに大量のコンテキストが含まれており、ドキュメントサイズも大きいため、チャンキングを実施する。
- ドキュメントは、技術要素ごとに明確なセクションわけがされており、各セクションで見るとLLMが扱えないレベルのデータサイズではない。そのため、セクション単位でチャンキングする。
- 各チャンキングのContentはEmbeddingする。
- Overlapping は行わない。
- 広い意味のコンテキストを保持するために、上位2つのヘッダー（Markdown形式：#, ##）をメタデータとして保持する。例えば、検索インデックスに関する記載があった場合に、それが「キーワード検索」に属する情報なのか、「ベクトル検索」に属する情報なのかを判断するために保持する。

In [ ]:
! pip install azure-search-documents==11.6.0b4
! pip install openai python-dotenv azure-identity cohere azure-ai-vision-imageanalysis
! pip install azure-storage-blob

In [ ]:
import os

from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient, SearchIndexerClient
from azure.search.documents.indexes.models import (
    AIServicesVisionParameters,
    AIServicesVisionVectorizer,
    AIStudioModelCatalogName,
    AzureMachineLearningVectorizer,
    AzureOpenAIVectorizer,
    AzureOpenAIModelName,
    AzureOpenAIParameters,
    AzureOpenAIEmbeddingSkill,
    BlobIndexerDataToExtract,
    BlobIndexerParsingMode,
    CognitiveServicesAccountKey,
    DefaultCognitiveServicesAccount,
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    FieldMapping,
    HnswAlgorithmConfiguration,
    HnswParameters,
    IndexerExecutionStatus,
    IndexingParameters,
    IndexingParametersConfiguration,
    InputFieldMappingEntry,
    KeyPhraseExtractionSkill,
    OutputFieldMappingEntry,
    ScalarQuantizationCompressionConfiguration,
    ScalarQuantizationParameters,
    SearchField,
    SearchFieldDataType,
    SearchIndex,
    SearchIndexer,
    SearchIndexerDataContainer,
    SearchIndexerDataIdentity,
    SearchIndexerDataSourceConnection,
    SearchIndexerIndexProjections,
    SearchIndexerIndexProjectionSelector,
    SearchIndexerIndexProjectionsParameters,
    SearchIndexerSkillset,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    SplitSkill,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
    VisionVectorizeSkill
)
from azure.search.documents.models import (
    HybridCountAndFacetMode,
    HybridSearch,
    SearchScoreThreshold,
    VectorizableTextQuery,
    VectorizableImageBinaryQuery,
    VectorizableImageUrlQuery,
    VectorSimilarityThreshold,
)
from azure.storage.blob import BlobServiceClient
from dotenv import load_dotenv
from IPython.display import Image, display, HTML
from openai import AzureOpenAI

In [ ]:
# Load environment variables
load_dotenv()

# Configuration
AZURE_AI_VISION_API_KEY = os.getenv("AZURE_AI_VISION_API_KEY")
AZURE_AI_VISION_ENDPOINT = os.getenv("AZURE_AI_VISION_ENDPOINT")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
BLOB_CONNECTION_STRING = os.getenv("BLOB_CONNECTION_STRING")
INDEX_NAME = "rag-search-index-pull-0621"
AZURE_SEARCH_ADMIN_KEY = os.getenv("AZURE_SEARCH_ADMIN_KEY")
AZURE_SEARCH_ENDPOINT = os.getenv("AZURE_SEARCH_ENDPOINT")

In [ ]:
# User-specified parameter
USE_AAD_FOR_SEARCH = False  # Set this to False to use API key for authentication

def authenticate_azure_search(api_key=None, use_aad_for_search=False):
    if use_aad_for_search:
        print("Using AAD for authentication.")
        credential = DefaultAzureCredential()
    else:
        print("Using API keys for authentication.")
        if api_key is None:
            raise ValueError("API key must be provided if not using AAD for authentication.")
        credential = AzureKeyCredential(api_key)
    return credential

azure_search_credential = authenticate_azure_search(api_key=AZURE_SEARCH_ADMIN_KEY, use_aad_for_search=USE_AAD_FOR_SEARCH)

### データセットのアップロード
Azure AI Search のインデクサーにより、データをインポートする場合は各データソースにデータを格納しておく必要がある。

In [ ]:
from utils.azure_blob_operation import upload_folder_to_blob

connection_string = BLOB_CONNECTION_STRING
container_name = "rag-knowledge-01-aisearch"
local_folder_path = "../output/"

upload_folder_to_blob(connection_string, container_name, local_folder_path)

### Create a blob data source connector on Azure AI Search

In [ ]:
def create_or_update_data_source(indexer_client, container_name, connection_string, index_name):
    """
    Create or update a data source connection for Azure AI Search.
    """
    container = SearchIndexerDataContainer(name=container_name)
    data_source_connection = SearchIndexerDataSourceConnection(
        name=f"{index_name}-blob",
        type="azureblob",
        connection_string=connection_string,
        container=container
    )
    try:
        indexer_client.create_or_update_data_source_connection(data_source_connection)
        print(f"Data source '{index_name}-blob' created or updated successfully.")
    except Exception as e:
        raise Exception(f"Failed to create or update data source due to error: {e}")

# Create a SearchIndexerClient instance
indexer_client = SearchIndexerClient(AZURE_SEARCH_ENDPOINT, azure_search_credential)

# Call the function to create or update the data source
create_or_update_data_source(indexer_client, container_name, BLOB_CONNECTION_STRING, INDEX_NAME)

## Create a search index

In [ ]:
def create_fields():
    """Creates the fields for the search index based on the specified schema."""
    return [
        SimpleField(
            name="id", type=SearchFieldDataType.String, key=True, filterable=True
        ),
        SearchField(name="content", type=SearchFieldDataType.String, searchable=True, analyzer_name="ja.microsoft"),
        SearchField(
			name="content_vector",
			type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
			vector_search_dimensions=1536,
			vector_search_profile_name="myHnswProfile",
			stored=False,
		),
        SearchField(name="metadata", type=SearchFieldDataType.String, searchable=True, analyzer_name="ja.microsoft"),
        SearchField(name="title", type=SearchFieldDataType.String, searchable=True, analyzer_name="ja.microsoft"),
    ]


def create_vector_search_configuration():
    """Creates the vector search configuration."""
    return VectorSearch(
        algorithms=[
            HnswAlgorithmConfiguration(
                name="myHnsw",
                parameters=HnswParameters(
                    m=4,
                    ef_construction=400,
                    ef_search=500,
                    metric=VectorSearchAlgorithmMetric.COSINE,
                ),
            )
        ],
        compressions=[
            ScalarQuantizationCompressionConfiguration(
                name="myScalarQuantization",
                rerank_with_original_vectors=True,
                default_oversampling=10,
                parameters=ScalarQuantizationParameters(quantized_data_type="int8"),
            )
        ],
        vectorizers=[
            AzureOpenAIVectorizer(
                name="myAzureOpenAIVectorizer",
                kind="azureOpenAI",
                azure_open_ai_parameters=AzureOpenAIParameters(
                    resource_uri=AZURE_OPENAI_ENDPOINT,
                    api_key=AZURE_OPENAI_API_KEY,
                    deployment_id="text-embedding-ada-002",
                    model_name=AzureOpenAIModelName.TEXT_EMBEDDING_ADA002,
                ),
            )
        ],
        profiles=[
            VectorSearchProfile(
                name="myHnswProfile",
                algorithm_configuration_name="myHnsw",
                compression_configuration_name="myScalarQuantization",
                vectorizer="myAzureOpenAIVectorizer",
            )
        ],
    )


def create_search_index(index_client, index_name, fields, vector_search):
    """Creates or updates a search index."""
    index = SearchIndex(
        name=index_name,
        fields=fields,
        vector_search=vector_search,
    )
    index_client.create_or_update_index(index=index)


index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT, credential=azure_search_credential
)
fields = create_fields()
vector_search = create_vector_search_configuration()

# Create the search index with the adjusted schema
create_search_index(index_client, INDEX_NAME, fields, vector_search)
print(f"Created index: {INDEX_NAME}")

In [ ]:
def create_search_index(index_name, azure_openai_endpoint, azure_openai_embedding_deployment_id, azure_openai_key=None):
    return SearchIndex(
        name=index_name,
        fields=[
            SearchField(
                name="chunk_id",
                type=SearchFieldDataType.String,
                key=True,
                hidden=False,
                filterable=True,
                sortable=True,
                facetable=False,
                searchable=True,
                analyzer_name="keyword"
            ),
            SearchField(
                name="parent_id",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=True,
                sortable=True,
                facetable=False,
                searchable=True
            ),
            SearchField(
                name="chunk",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
                searchable=True,
                analyzer_name="ja.microsoft" # replace with your analyzer
            ),
            SearchField(
                name="title",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
                searchable=True,
                analyzer_name="ja.microsoft" # replace with your analyzer
            ),
            SearchField(
                name="key_phrases",
                type=SearchFieldDataType.String,
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
                searchable=True,
                analyzer_name="ja.microsoft" # replace with your analyzer
            ),
            SearchField(
                name="vector",
                type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
                hidden=False,
                filterable=False,
                sortable=False,
                facetable=False,
                searchable=True,
                vector_search_dimensions=1536,
                vector_search_profile_name="profile"
            )
        ],
        vector_search=VectorSearch(
			algorithms=[
				HnswAlgorithmConfiguration(
					name="myHnsw",
					parameters=HnswParameters(
						m=4,
						ef_construction=400,
						ef_search=500,
						metric=VectorSearchAlgorithmMetric.COSINE,
					),
				)
			],
			compressions=[
				ScalarQuantizationCompressionConfiguration(
					name="myScalarQuantization",
					rerank_with_original_vectors=True,
					default_oversampling=10,
					parameters=ScalarQuantizationParameters(quantized_data_type="int8"),
				)
			],
			vectorizers=[
				AzureOpenAIVectorizer(
					name="myAzureOpenAIVectorizer",
					kind="azureOpenAI",
					azure_open_ai_parameters=AzureOpenAIParameters(
						resource_uri=azure_openai_endpoint,
						api_key=azure_openai_key,
						deployment_id=azure_openai_embedding_deployment_id,
						model_name=AzureOpenAIModelName.TEXT_EMBEDDING_ADA002,
					),
				)
			],
			profiles=[
				VectorSearchProfile(
					name="profile",
					algorithm_configuration_name="myHnsw",
					compression_configuration_name="myScalarQuantization",
					vectorizer="myAzureOpenAIVectorizer",
				)
			],
    	)
	)

index = create_search_index(
    INDEX_NAME,
    AZURE_OPENAI_ENDPOINT,
    "text-embedding-ada-002", # replace with your deployment name
    AZURE_OPENAI_API_KEY
)
index_client = SearchIndexClient(
    endpoint=AZURE_SEARCH_ENDPOINT, credential=azure_search_credential
)
index_client.create_or_update_index(index)

print(f"Created index: {INDEX_NAME}")

## Create a Skillset

In [ ]:
def create_search_skillset(
        skillset_name,
        index_name,
        azure_openai_endpoint,
        azure_openai_embedding_deployment_id,
        azure_openai_key=None,
        text_split_mode='pages',
        maximum_page_length=2000,
        page_overlap_length=500):
    return SearchIndexerSkillset(
        name=skillset_name,
        skills=[
            SplitSkill(
                name="Text Splitter",
                default_language_code="ja", # replace documents language code
                text_split_mode=text_split_mode,
                maximum_page_length=maximum_page_length,
                page_overlap_length=page_overlap_length,
                context="/document",
                inputs=[
                    InputFieldMappingEntry(
                        name="text",
                        source="/document/content"
                    )
                ],
                outputs=[
                    OutputFieldMappingEntry(
                        name="textItems",
                        target_name="pages"
                    )
                ]
            ),
            AzureOpenAIEmbeddingSkill(
                name="Embeddings",
                resource_uri=azure_openai_endpoint,
                deployment_id=azure_openai_embedding_deployment_id,
                api_key=azure_openai_key, # Optional if using RBAC authentication
                model_name=AzureOpenAIModelName.TEXT_EMBEDDING_ADA002,
                context="/document/pages/*",
                inputs=[
                    InputFieldMappingEntry(
                        name="text",
                        source="/document/pages/*"
                    )
                ],
                outputs=[
                    OutputFieldMappingEntry(
                        name="embedding",
                        target_name="vector"
                    )
                ]
            ),
            KeyPhraseExtractionSkill(
				name="Key Phrase Extraction",
				context="/document/pages/*",
				default_language_code="ja", # replace documents language code
                inputs=[
                    InputFieldMappingEntry(
                        name="text",
                        source="/document/pages/*"
                    )
                ],
                outputs=[
                    OutputFieldMappingEntry(
                        name="keyPhrases",
                        target_name="key_phrases"
                    )
                ]
			)
        ],
        index_projections=SearchIndexerIndexProjections(
            selectors=[
                SearchIndexerIndexProjectionSelector(
                    target_index_name=index_name,
                    parent_key_field_name="parent_id",
                    source_context="/document/pages/*",
                    mappings=[
                        InputFieldMappingEntry(
                            name="chunk",
                            source="/document/pages/*"
                        ),
                        InputFieldMappingEntry(
                            name="vector",
                            source="/document/pages/*/vector"
                        ),
                        InputFieldMappingEntry(
                            name="title",
                            source="/document/metadata_storage_name"
                        ),
                        InputFieldMappingEntry(
                            name="key_phrases",
                            source="/document/pages/*/key_phrases"
                        ),
                    ]
                )
            ],
            parameters=SearchIndexerIndexProjectionsParameters(projection_mode="skipIndexingParentDocuments")
        )
    )

skillset_name = f"{INDEX_NAME}-skillset"
skillset = create_search_skillset(
    skillset_name,
    INDEX_NAME,
    AZURE_OPENAI_ENDPOINT,
    "text-embedding-ada-002", # replace with your deployment name
    AZURE_OPENAI_API_KEY,
    text_split_mode='pages',
    maximum_page_length=2000,
    page_overlap_length=500
)
search_indexer_client = SearchIndexerClient(endpoint=AZURE_SEARCH_ENDPOINT, credential=azure_search_credential)
search_indexer_client.create_or_update_skillset(skillset)

## Run Indexer

In [ ]:
def create_search_indexer(indexer_name, skillset_name, datasource_name, index_name):
    return SearchIndexer(
        name=indexer_name,
        data_source_name=datasource_name,
        target_index_name=index_name,
        skillset_name=skillset_name
    )

indexer_name = f"{INDEX_NAME}-indexer"
indexer = create_search_indexer(indexer_name, skillset_name, f"{INDEX_NAME}-blob", INDEX_NAME)


In [ ]:
search_indexer_client.create_or_update_indexer(indexer)
print(f"{indexer_name} created or updated.")
search_indexer_client.run_indexer(indexer_name)
print(f"{indexer_name} is running. If queries return no results, please wait a bit and try again.")

# 04_Query-Design
Tips for Azure AI Search Query-Design

### Simple Search
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、フルテキスト検索＋ベクトル検索を組み合わせたHybrid検索を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- クエリはユーザのクエリをそのまま検索インデックスのクエリに利用する。

In [ ]:
search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT, index_name=INDEX_NAME, credential=azure_search_credential)

In [ ]:
query="ベクトル検索時の設定要素について教えてください"

vector_query = VectorizableTextQuery(
    text=query,
    k_nearest_neighbors=50,
    fields="vector",
)

# Perform the search
results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    top=5,
    select="chunk, title, key_phrases",
	search_fields=["chunk", "title", "key_phrases"],
)

for result in results:
    print(result)

### Query Extensions
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、フルテキスト検索＋ベクトル検索を組み合わせたHybrid検索を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- また、ユーザクエリから検索クエリを新しく生成する。
  - クエリはユーザのクエリをスタンドアローンなクエリに変換する。
  - また、検索のカバレッジを大きくするために、類似した入力クエリを複数生成する。

In [ ]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)

system_message = """
# Your Task
- Given the following conversation history and the users next question,rephrase the question to be a stand alone question.
- You also need to extend the original question to generate 5 related queries. This is done to capture the broader context of the user's question.
- You must output json format. In other words, You must output array of questions that length is 5.

# Json format example:
{
	"questions": [
		"related question 1",
		"related question 2",
		"related question 3",
		"related question 4",
		"related question 5"
	]
}
"""

def generate_expanded_query(text):
    message_text = [
		{"role":"system","content": system_message},
		{"role":"user","content": text}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content


In [ ]:
query="ベクトル検索時の設定要素について教えてください"
expanded_query = generate_expanded_query(query)
parsed_data = json.loads(expanded_query)
parsed_data

In [ ]:
for question in parsed_data["questions"]:
	vector_query = VectorizableTextQuery(
		text=question,
		k_nearest_neighbors=50,
		fields="vector",
	)
	# Perform the search
	results = search_client.search(
		search_text=query,
		vector_queries=[vector_query],
		top=5,
		select="chunk, title, key_phrases",
		search_fields=["chunk", "title", "key_phrases"],
	)
	print("query: ", question)
	for result in results:
		print(result)
	print("\n")

### HyDE (Hypothetical Document Embeddings)
- ユーザのクエリのコンテキストをとらえたい、かつサービスに特化したワードがクエリに含まれる可能性が高いため、フルテキスト検索＋ベクトル検索を組み合わせたHybrid検索を採用する。
  - Hybrid検索のスコアはAzure AI Searchでは、Reciprocal Rank Fusion (RRF) が採用される。 
- また、ユーザクエリから検索クエリを新しく生成する。
  - クエリはユーザのクエリをスタンドアローンなクエリに変換する。
  - また、ユーザのクエリに基づいて仮想的な応答をLLMで作成し、それをベクトル変換した結果を用いて検索をかけるHyDEを採用します。クエリを検索対象のベクトルにより近いものに変換することで、検索精度を高めることを狙った手法です。
  - HyDEはLLMがまったく知識を持たないような領域だと役に立たない可能性があるため採用する際は注意してください。

In [ ]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)


def generate_hypothetical_query(text):
    hypothetical_gen_instruction = f"""Please write a passage to answer the question
	Question: {text}
	Passage:
	"""
    message_text = [
		{"role":"system","content": "You are an AI assistant."},
		{"role":"user","content": hypothetical_gen_instruction}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		# response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content

In [ ]:
query="ベクトル検索時の設定要素について教えてください"
hypothetical_answer = generate_hypothetical_query(query)
hypothetical_answer

In [ ]:
vector_query = VectorizableTextQuery(
	text=hypothetical_answer,
	k_nearest_neighbors=50,
	fields="vector",
)
# Perform the search
results = search_client.search(
	search_text=hypothetical_answer,
	vector_queries=[vector_query],
	top=5,
	select="chunk, title, key_phrases",
	search_fields=["chunk", "title", "key_phrases"],
)
for result in results:
	print(result)

## 05_Generate-Answer
検索インデックスから取得したものをコンテキストとして与えて、それをベースにした回答を生成させるプロンプトを設定します。
プロンプトエンジニアリングに関する包括的なガイダンスは以下を参照ください。

https://learn.microsoft.com/ja-jp/azure/ai-services/openai/concepts/prompt-engineering

In [ ]:
import re
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-02-01"
)


def generate_answer(query, context):
    system_message = f"""
    system:
	You are an AI assistant that helps users answer questions given a specific context. You will be given a context and asked a question based on that context. Your answer should be as precise as possible and should only come from the context.
	Please add citation after each sentence when possible in a form "(Source: citation)". 
	context: {context}
	user: 
	"""
    message_text = [
		{"role":"system","content": system_message},
		{"role":"user","content": query}
	]
    completion = client.chat.completions.create(
		model="gpt-4o", # model = "deployment_name"
		messages = message_text,
		# response_format={"type": "json_object"},
		temperature=0,
		)
    return completion.choices[0].message.content

In [ ]:
context_text = ""
for result in results:
	context_text += result["chunk"] + " "

answer = generate_answer(query, context_text)
answer

## 06_Evaluation
RAG の Evaluation は、「検索評価」と「生成評価」にわけて実施することが推奨される。

### 検索評価 - 簡易評価
- シンプルな検索評価として、検索結果の上位5件にユーザクエリを解決するための情報が含まれているかどうかを評価します。
- クエリの種類は、想定されるエンドユーザーのクエリや、異なるドキュメントを答えとなるようなクエリを複数パターン用意します。
  - 回答に複数の文が必要な抽象的な質問：
  - 検索エンジンに一般的に入力されるものと同様の短縮されたクエリ：
  - 回答が質問とは異なる単語やフレーズを使用しているクエリ：
  - 回答が 1 つしかないクエリ
  - 複数の内容を質問しているクエリ
- ここではサンプルのため5パターンのクエリを用意しますが、包括的な評価をするためには、100件以上のパターンを用意することが推奨されます。（もちろん、ドキュメントの量やユーザのタスクによって異なるため、それぞれの要件にあわせて設計が必要です）
  - [参考：評価用データセットの作成](https://github.com/microsoft/promptflow-resource-hub/blob/main/sample_gallery/golden_dataset/copilot-golden-dataset-creation-guidance.md)


In [ ]:
# Test query set
queries = [
	"AI Search について勉強しています。ベクトル検索時の設定要素について教えてください",
	"ハイブリッド検索　メリット",
	"Azure AI Search には、リランクのモデルが利用できるか？",
	"フルテキスト検索の取得は最大何件か",
	"Hybrid検索とセマンティックランカーの特徴と違いはなんですか？"
]

#### シンプルクエリ

クエリを実行

In [ ]:
import pprint

# Test for simple query
for query in queries:
    vector_query = VectorizableTextQuery(
        text=query,
        k_nearest_neighbors=50,
        fields="vector",
    )

    # Perform the search
    results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        top=5,
        select="chunk, title, key_phrases",
        search_fields=["chunk", "title", "key_phrases"],
    )

    print("Query:")
    pprint.pprint(query)
    print("\nResults:")
    for result in results:
        pprint.pprint(result)
        print("\n")

評価を実施
1. 回答に複数の文が必要な抽象的な質問：
	- query: AI Search について勉強しています。ベクトル検索時の設定要素について教えてください
	- result: 合致するドキュメントが上位5番目までに含まれない（ベクトル検索についての説明は含まれるが、AI Search での設定要素に関する説明が足りていない）
2. 検索エンジンに一般的に入力されるものと同様の短縮されたクエリ：
	- query: ハイブリッド検索　メリット
	- result: 上位1番目のドキュメントが内容に合致
3. 回答が質問とは異なる単語やフレーズを使用しているクエリ：
	- query: Azure AI Search には、リランクのモデルが利用できるか？
	- result: 上位3番目のドキュメントが内容に合致
4. 回答が 1 つしかないクエリ：
	- query: フルテキスト検索の取得は最大何件か
	- result: 上位1番目のドキュメントが内容に合致
5. キーワードクエリ：
	- query: Hybrid検索とセマンティックランカーの特徴と違いはなんですか？
	- result: 上位1, 5番目のドキュメントが内容に合致

#### HyDE

クエリを実行

In [ ]:
import pprint

# Test for simple query
for query in queries:
    hypothetical_answer = generate_hypothetical_query(query)   
    vector_query = VectorizableTextQuery(
		text=hypothetical_answer,
		k_nearest_neighbors=50,
		fields="vector",
	)

    # Perform the search
    results = search_client.search(
        search_text=hypothetical_answer,
        vector_queries=[vector_query],
        top=5,
        select="chunk, title, key_phrases",
        search_fields=["chunk", "title", "key_phrases"],
    )

    print("Query:")
    pprint.pprint(query)
    print("\nHyDE Query:")
    pprint.pprint(hypothetical_answer)
    print("\nResults:")
    for result in results:
        pprint.pprint(result)
        print("\n")

評価を実施
1. 回答に複数の文が必要な抽象的な質問：
	- query: AI Search について勉強しています。ベクトル検索時の設定要素について教えてください
	- result: 上位1番目のドキュメントが内容に合致
2. 検索エンジンに一般的に入力されるものと同様の短縮されたクエリ：
	- query: ハイブリッド検索　メリット
	- result: 上位1番目のドキュメントが内容に合致
3. 回答が質問とは異なる単語やフレーズを使用しているクエリ：
	- query: Azure AI Search には、リランクのモデルが利用できるか？
	- result: 上位1番目のドキュメントが内容に合致
4. 回答が 1 つしかないクエリ：
	- query: フルテキスト検索の主とk数は最大何件か
	- result: 上位1番目のドキュメントが内容に合致
5. キーワードクエリ：
	- query: Hybrid検索とセマンティックランカーの特徴と違いはなんですか？
	- result: 上位1, 5番目のドキュメントが内容に合致

### 検索評価 - 厳密な評価
TBD
- 評価指標の設計
- クエリパターン
- 評価用データの選定・設計

### 生成評価 - 簡易評価
- シンプルな検索評価として、検索結果の上位5件にユーザクエリを解決するための情報が含まれているかどうかを評価します。
- クエリの種類は、想定されるエンドユーザーのクエリや、異なるドキュメントを答えとなるようなクエリを複数パターン用意します。
  - 回答に複数の文が必要な抽象的な質問：
  - 検索エンジンに一般的に入力されるものと同様の短縮されたクエリ：
  - 回答が質問とは異なる単語やフレーズを使用しているクエリ：
  - 回答が 1 つしかないクエリ
  - 複数の内容を質問しているクエリ
- ここではサンプルのため5パターンのクエリを用意しますが、包括的な評価をするためには、100件以上のパターンを用意することが推奨されます。（もちろん、ドキュメントの量やユーザのタスクによって異なるため、それぞれの要件にあわせて設計が必要です）
  - [参考：評価用データセットの作成](https://github.com/microsoft/promptflow-resource-hub/blob/main/sample_gallery/golden_dataset/copilot-golden-dataset-creation-guidance.md)


In [ ]:
# Define RAG Pipeline
def rag_pipeline_with_hyde(query):
	hypothetical_answer = generate_hypothetical_query(query)
	vector_query = VectorizableTextQuery(
		text=hypothetical_answer,
		k_nearest_neighbors=50,
		fields="vector",
	)
	# Perform the search
	results = search_client.search(
		search_text=hypothetical_answer,
		vector_queries=[vector_query],
		top=5,
		select="chunk, title, key_phrases",
		search_fields=["chunk", "title", "key_phrases"],
	)
	
	context_text = ""
	for result in results:
		context_text += result["chunk"] + " "

	return generate_answer(query, context_text)

In [ ]:
# Test query set
queries = [
	"AI Search について勉強しています。ベクトル検索時の設定要素について教えてください",
	"ハイブリッド検索　メリット",
	"Azure AI Search には、リランクのモデルが利用できるか？",
	"フルテキスト検索の取得は最大何件か",
	"Hybrid検索とセマンティックランカーの特徴と違いはなんですか？"
]

In [ ]:
import time

for query in queries:
    start_time = time.time()
    answer = rag_pipeline_with_hyde(query)
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Query: {query}")
    print(f"Answer: {answer}")
    print(f"Elapsed time: {elapsed_time} seconds")
    print("\n")

In [ ]:
# Test query set
queries = [
	"AI Search において検索結果をハイライトする際に、単語レベルで設定することが可能か"
]

In [ ]:
import time

for query in queries:
    start_time = time.time()
    answer = rag_pipeline_with_hyde(query)
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Query: {query}")
    print(f"Answer: {answer}")
    print(f"Elapsed time: {elapsed_time} seconds")
    print("\n")